To query the hvac30d
data

In [1]:
from riak import RiakClient, RiakNode

/home/stephen/ESL/local/lib/python2.7/site-packages/riak/security.py:45: UserWarning: OpenSSL 1.0.1f 6 Jan 2014 (>= 1.0.1g required), TLS 1.2 support: False
  warnings.warn(msg, UserWarning)


In [2]:
myClient = RiakClient(pb_port=8087, protocol='pbc')

In [3]:
myClient.ping()

True

In [4]:
myClient.ts_describe('hvac30d')

Preliminary data exploration using pandas to get max min values and ranges that can be queried.

In [7]:
import pandas as pd

In [10]:
df1=[]
df1=pd.read_csv('HVAC-ms-headings.csv')

In [11]:
df1.describe()

,Timestamp,TargetTemp,ActualTemp,SysID,SysAge,BldgID
count,8.000000e+03,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,1.371339e+12,67.499625,67.636875,10.382125,15.349750,10.509000
std,7.481817e+08,1.708981,7.452010,5.731641,8.686173,5.770969
min,1.370045e+12,65.000000,55.000000,1.000000,1.000000,1.000000
25%,1.370691e+12,66.000000,61.000000,5.000000,8.000000,6.000000
50%,1.371339e+12,67.000000,68.000000,10.000000,15.000000,10.000000
75%,1.371986e+12,69.000000,74.000000,15.000000,23.000000,16.000000
max,1.372636e+12,70.000000,80.000000,20.000000,30.000000,20.000000


In [5]:
#18 days worth
qs="""
    select * from hvac30d WHERE  readingtime > 1370044800000 and readingtime < 1371600001000
     and buildingid = '1' 
    and systemid= '1'
"""
ds=myClient.ts_query('hvac30d',qs)
print ds
for r in ds.rows:
    print r


['1', '1', datetime.datetime(2013, 6, 1, 3, 33, 7), 7L, 67L, 62L]
['1', '1', datetime.datetime(2013, 6, 1, 16, 45, 56), 6L, 68L, 69L]
['1', '1', datetime.datetime(2013, 6, 2, 17, 0, 1), 29L, 68L, 63L]
['1', '1', datetime.datetime(2013, 6, 7, 21, 13, 19), 27L, 66L, 62L]
['1', '1', datetime.datetime(2013, 6, 8, 1, 13, 19), 3L, 70L, 60L]
['1', '1', datetime.datetime(2013, 6, 11, 12, 43, 51), 20L, 69L, 77L]
['1', '1', datetime.datetime(2013, 6, 14, 6, 43, 51), 8L, 69L, 78L]


In [18]:
#60 days worth
qs="""
    select * from hvac30d WHERE  readingtime > 1370044800000 and readingtime < 1375228801000
     and buildingid = '2' 
    and systemid= '2'
"""
ds=myClient.ts_query('hvac30d',qs)
print ds
#print ds.rows
for r in ds.rows:
    print r

['2', '2', datetime.datetime(2013, 6, 1, 16, 43, 51), 15L, 66L, 76L]
['2', '2', datetime.datetime(2013, 6, 1, 23, 0, 1), 13L, 67L, 56L]
['2', '2', datetime.datetime(2013, 6, 2, 11, 33, 7), 22L, 66L, 77L]
['2', '2', datetime.datetime(2013, 6, 6, 23, 43, 51), 9L, 66L, 78L]
['2', '2', datetime.datetime(2013, 6, 7, 0, 13, 20), 12L, 67L, 80L]
['2', '2', datetime.datetime(2013, 6, 8, 3, 13, 19), 13L, 65L, 68L]
['2', '2', datetime.datetime(2013, 6, 9, 10, 45, 56), 29L, 65L, 71L]
['2', '2', datetime.datetime(2013, 6, 9, 14, 13, 19), 17L, 67L, 68L]
['2', '2', datetime.datetime(2013, 6, 11, 11, 13, 19), 3L, 66L, 60L]
['2', '2', datetime.datetime(2013, 6, 12, 11, 43, 51), 20L, 65L, 73L]
['2', '2', datetime.datetime(2013, 6, 14, 21, 45, 56), 22L, 70L, 76L]
['2', '2', datetime.datetime(2013, 6, 16, 22, 43, 51), 15L, 70L, 77L]
['2', '2', datetime.datetime(2013, 6, 17, 7, 33, 7), 11L, 66L, 75L]
['2', '2', datetime.datetime(2013, 6, 17, 22, 13, 19), 4L, 67L, 73L]
['2', '2', datetime.datetime(2013, 6, 

System Age appears to be very anomalous, it looks like junk.

In [22]:
td=myClient.ts_describe('hvac30d')
for c in td.columns:
    print c
for r in td.rows:
    print r

('Column', 0)
('Type', 0)
('Is Null', 4)
('Primary Key', 1)
('Local Key', 1)
['buildingid', 'varchar', False, 1L, 1L]
['systemid', 'varchar', False, 2L, 2L]
['readingtime', 'timestamp', False, 3L, 3L]
['systemage', 'sint64', False, None, None]
['targettemp', 'sint64', False, None, None]
['actualtemp', 'sint64', False, None, None]


In [25]:
print len(td.rows)

6


In [28]:
qs="""
    select COUNT(*) from hvac30d WHERE  readingtime > 1370044800000 and readingtime < 1375228801000
     and buildingid = '2' 
    and systemid= '2'
"""
ds=myClient.ts_query('hvac30d',qs)
print ds
print ds.rows
for r in ds.rows:
    print r

[[24L]]
[24L]


In [36]:
qs="""
    select * from hvac30d WHERE  readingtime > 1370044800000 and readingtime < 1375228801000
     and buildingid = '2' 
    and systemid= '3' 
    or 
    readingtime > 1370044800000 and readingtime < 1375228801000
     and buildingid = '2' 
     and systemid='2'
"""
ds=myClient.ts_query('hvac30d',qs)
print ds
print ds.rows
for r in ds.rows:
    print r

RiakError: 'incomplete_where_clause: Neither upper or lower time bounds were specified in the query.'